In [14]:
import cantera as ct
import timeit
import pandas as pd
import csv
import glob
import os


mech = 'Konnov_Han2021_dup_repairedfN2.yaml' # including NOx

gas = ct.Solution(mech)

# Simulation parameters 
Pressure = 15*ct.one_atm   # pressure [Pa]
Temperature = 500     # unburned gas temperature [K]   
phi = 1.0         # Equivalence ratio

# Solution
gas.TP = Temperature, Pressure
gas.set_equivalence_ratio(phi, 'H2:1.0', 'O2:1.0, N2:3.76') # Air
#gas.set_equivalence_ratio(phi, 'H2:1.0', 'N2O:1.0') # N2O


# Flame object
Laminar1Dflame = ct.FreeFlame(gas, width=0.1)
Laminar1Dflame.set_refine_criteria(ratio=3, slope=0.002, curve=0.002, prune=0.9*0.002)  # Ensuring 4000 grids
Laminar1Dflame.set_max_grid_points(Laminar1Dflame.flame, 1e4)
Laminar1Dflame.max_time_step_count = 1e6
Laminar1Dflame.transport_model = 'Multi'
Laminar1Dflame.soret_enabled = True
Laminar1Dflame.solve(loglevel=0, refine_grid=True, auto=False)


# Total HRR
csvFile = open ('heat_release_{}_{}_{}_{}.csv'.format(mech,Temperature, Pressure, phi), 'w')
writer = csv.writer(csvFile)
writer.writerow(list(Laminar1Dflame.grid))
writer.writerow(list(Laminar1Dflame.heat_release_rate))
csvFile.close()
forTransposing = pd.read_csv('heat_release_{}_{}_{}_{}.csv'.format(mech,Temperature,Pressure,phi))
csvfile_transposed = forTransposing.T
csvfile_transposed.to_csv('heat_release_{}_{}_{}_{}.csv'.format(mech,Temperature,Pressure,phi))

# HRR for each reaction
csvFile = open ('heat_production_rates_{}_{}_{}_{}.csv'.format(mech,Temperature,Pressure,phi), 'w')
writer = csv.writer(csvFile)
writer.writerow(['Reaction Equations     /    z (m)']+list(Laminar1Dflame.grid))
for i in range (len(Laminar1Dflame.heat_production_rates)):      
 writer.writerow([gas.reaction_equation(i)]+list(Laminar1Dflame.heat_production_rates[i]))
csvFile.close()
forTransposing = pd.read_csv('heat_production_rates_{}_{}_{}_{}.csv'.format(mech,Temperature,Pressure,phi))
csvfile_transposed = forTransposing.T
csvfile_transposed.to_csv('heat_production_rates_{}_{}_{}_{}.csv'.format(mech,Temperature,Pressure,phi))

data = pd.read_csv('heat_release_{}_{}_{}_{}.csv'.format(mech,Temperature,Pressure,phi))
HRR = data[1]

reactions = pd.read_csv('heat_production_rates_{}_{}_{}_{}.csv'.format(mech,Temperature,Pressure,phi))
react1 = reactions['N + NO <=> N2 + O']
react2 = reactions['N + OH <=> H + NO']
react3 = reactions['H2O + H2O <=> H + OH + H2O']

# Plot
plt.plot(Laminar1Dflame.grid, react1, label = "N + NO <=> N2 + O")
plt.plot(Laminar1Dflame.grid, react2, label = "N + OH <=> H + NO")
plt.plot(Laminar1Dflame.grid, react2, label = 'H2O + H2O <=> H + OH + H2O')
plt.plot(Laminar1Dflame.grid, HRR, label = 'Total HRR')
plt.grid(True)
plt.xlabel("Burner Height [m]")
plt.ylabel("HRR [W/$m^2$]")
pylab.xlim(0.03, 0.1)
pylab.ylim(-3e7, 3e7)


C:\Users\schurrj1\AppData\Local\Temp\ipykernel_25848\1657888569.py:49: DeprecationWarning: 'reaction_equation' is deprecated and will be removed after Cantera 2.6.
Replaceable by property 'equation' of the corresponding reaction object.
  writer.writerow([gas.reaction_equation(i)]+list(Laminar1Dflame.heat_production_rates[i]))


KeyError: 1

In [13]:
csvFile = open('heat_production_rates_Konnov_Han2021_dup_repairedfN2.yaml_500_1519875.0_1.0.csv'.format(mech,Temperature,Pressure,phi),'w')
writer = csv.writer(csvFile)
writer.writerow(['Reaction Equations     /    z (m)']+list(Laminar1Dflame.grid))

90018